In [1]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
from sklearn.metrics import log_loss
import sys


import os
import pandas as pd

import numpy as np
from glob import glob
import cv2
import skimage
from skimage.transform import resize

import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.models import load_model
import keras.callbacks as kcall
from keras.optimizers import Adam
from keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
import matplotlib.pyplot as plt
from keras.applications.resnet50 import ResNet50
from keras import optimizers, metrics, models
from keras.layers import Input, Flatten, Dense
from keras.optimizers import SGD, Adam, rmsprop

%matplotlib inline

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [11]:
import keras.backend as K
from keras.optimizers import Optimizer


class Eve(Optimizer):
    '''Eve optimizer.

    Default parameters follow those provided in the original paper.

    # Arguments
        lr: float >= 0. Learning rate.
        beta_1/beta_2/beta_3: floats, 0 < beta < 1. Generally close to 1.
        small_k/big_K: floats
        epsilon: float >= 0. Fuzz factor.

    # References
        - [Improving Stochastic Gradient Descent With FeedBack](http://arxiv.org/abs/1611.01505v1.pdf)
    '''

    def __init__(self, lr=0.001, beta_1=0.9, beta_2=0.999,
                 beta_3=0.999, small_k=0.1, big_K=10,
                 epsilon=1e-8, decay=0., **kwargs):
        super(Eve, self).__init__(**kwargs)
        self.__dict__.update(locals())
        self.iterations = K.variable(0)
        self.lr = K.variable(lr)
        self.beta_1 = K.variable(beta_1)
        self.beta_2 = K.variable(beta_2)
        self.beta_3 = K.variable(beta_3)
        self.small_k = K.variable(small_k)
        self.big_K = K.variable(big_K)
        self.decay = K.variable(decay)
        self.inital_decay = decay

    def get_updates(self, params, loss):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]

        lr = self.lr
        if self.inital_decay > 0:
            lr *= (1. / (1. + self.decay * self.iterations))

        t = self.iterations + 1
        lr_t = lr * K.sqrt(1. - K.pow(self.beta_2, t)) / (1. - K.pow(self.beta_1, t))

        shapes = [K.get_variable_shape(p) for p in params]
        ms = [K.zeros(shape) for shape in shapes]
        vs = [K.zeros(shape) for shape in shapes]
        f = K.variable(0)
        d = K.variable(1)
        self.weights = [self.iterations] + ms + vs + [f, d]

        cond = K.greater(t, K.variable(1))
        small_delta_t = K.switch(K.greater(loss, f), self.small_k + 1, 1. / (self.big_K + 1))
        big_delta_t = K.switch(K.greater(loss, f), self.big_K + 1, 1. / (self.small_k + 1))

        c_t = K.minimum(K.maximum(small_delta_t, loss / (f + self.epsilon)), big_delta_t)
        f_t = c_t * f
        r_t = K.abs(f_t - f) / (K.minimum(f_t, f))
        d_t = self.beta_3 * d + (1 - self.beta_3) * r_t

        f_t = K.switch(cond, f_t, loss)
        d_t = K.switch(cond, d_t, K.variable(1.))

        self.updates.append(K.update(f, f_t))
        self.updates.append(K.update(d, d_t))

        for p, g, m, v in zip(params, grads, ms, vs):
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)
            p_t = p - lr_t * m_t / (d_t * K.sqrt(v_t) + self.epsilon)

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))

            new_p = p_t
            self.updates.append(K.update(p, new_p))
        return self.updates

    def get_config(self):
        config = {'lr': float(K.get_value(self.lr)),
                  'beta_1': float(K.get_value(self.beta_1)),
                  'beta_2': float(K.get_value(self.beta_2)),
                  'beta_3': float(K.get_value(self.beta_3)),
                  'small_k': float(K.get_value(self.small_k)),
                  'big_K': float(K.get_value(self.big_K)),
                  'epsilon': self.epsilon}
        base_config = super(Eve, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


In [10]:
print(os.listdir('../input/pretrained-models/pretrained-models'))

['xception_weights_tf_dim_ordering_tf_kernels_notop.h5']


In [12]:
## Intilizing variables
output_classes = 4
# batch_size = 32 
# epochs = 30

# sgd_opt = SGD(lr=1E-2, decay=1E-4, momentum=0.9, nesterov=True)
# adam_opt = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1E-4)
# eve_opt = Eve(lr=1E-4, decay=1E-4, beta_1=0.9, beta_2=0.999, beta_3=0.999, small_k=0.1, big_K=10, epsilon=1e-08)

xception_weights = '../input/pretrained-models/pretrained-models/xception_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [13]:
space = {
         'lr': hp.choice('lr',[0.001, 0.0001, 0.00001, 0.000001]),
#          'dropout': hp.choice('dropout', [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]),
         'batch_size': hp.choice('batch_size', [32, 64]),
         'epochs': hp.choice('epochs', [20]),
#        'optimizer': hp.choice('optimizer',['sgd','adam','rmsprop']),
#          'activation': hp.choice('activation',['relu',
#                                                 'tanh']),
        }

In [14]:
train_dir = '../input/iciarreinhardfull/reinhard/Reinhard'
for root,dirs,files in os.walk(train_dir):
    print (root, len(files))

../input/iciarreinhardfull/reinhard/Reinhard 0
../input/iciarreinhardfull/reinhard/Reinhard/InSitu 3500
../input/iciarreinhardfull/reinhard/Reinhard/Normal 3500
../input/iciarreinhardfull/reinhard/Reinhard/Invasive 3500
../input/iciarreinhardfull/reinhard/Reinhard/Benign 3500


In [15]:
def f_nn(params):   
    print ('Parameters testing: ', params)
    batch_size=params['batch_size']
    epochs=params['epochs']

    from keras.preprocessing.image import ImageDataGenerator

    # this is the augmentation configuration we will use for training
    train_datagen = ImageDataGenerator(
        rescale=1. / 255,
        shear_range=0.2,
        zoom_range=0.2,
            width_shift_range=0.2,
            height_shift_range=0.2,
            fill_mode='nearest',
        horizontal_flip=True,
        validation_split=0.25)

    # this is the augmentation configuration we will use for testing:
    # only rescaling
    test_datagen = ImageDataGenerator(rescale=1. / 255)


    #target_size: Tuple of integers (height, width), default: (256, 256). 
    #The dimensions to which all images found will be resized.
    target_size = (256, 256)
    #target_size = (height, width)
    train_generator = train_datagen.flow_from_directory(
            train_dir,
            target_size = target_size,       
            class_mode = 'categorical',
            batch_size=32,
            subset="training",
            shuffle = True)

    validation_generator = train_datagen.flow_from_directory(
            train_dir,
            target_size = target_size,        
            class_mode = 'categorical',
            batch_size=32,
            subset = "validation",
            shuffle = True)

    
    model = Sequential()

    model.add(Xception(weights = xception_weights , include_top=False, pooling = 'avg'))
    model.add(Dense(units=output_classes, activation='softmax'))


    model.compile(loss = 'categorical_crossentropy',
                  optimizer = Eve(lr=params['lr'], decay=1E-4, beta_1=0.9, beta_2=0.999, beta_3=0.999, small_k=0.1, big_K=10, epsilon=1e-08),
                  metrics = ['accuracy'])

    history = model.fit_generator(
      train_generator,
      steps_per_epoch = 2000,
      epochs = epochs,
      validation_data = validation_generator,
#       validation_steps = 100,
      verbose = 1, callbacks=get_callbacks(params))
    
    
    best_epoch = np.argmax(history.history['val_acc'])
    best_val_acc = np.max(history.history['val_acc'])
    print('Epoch {} - val acc: {}'.format(best_epoch, best_val_acc))
    sys.stdout.flush() 
    
    return {'val_acc': best_val_acc, 'best_epoch': best_epoch, 'eval_time': time.time(), 'status': STATUS_OK}

In [16]:
output_classes = 4

In [17]:
def get_callbacks(params):
    callbacks =[EarlyStopping(monitor='val_acc', patience=5, verbose=1),
                ModelCheckpoint('callbacks/{}.h5'.format(params['batch_size']), save_best_only=True),
             TensorBoard('tensor-logs/logs-gridsearch', write_graph=True, write_grads=True, write_images=True, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)]
    return callbacks

____

In [ ]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals=30, trials=trials)
print(best)

Parameters testing:  {'batch_size': 32, 'epochs': 20, 'lr': 0.001}
